In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/scrapped-lyrics-from-6-genres/artists-data.csv
/kaggle/input/scrapped-lyrics-from-6-genres/lyrics-data.csv


In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer,AdamW,get_linear_schedule_with_warmup
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [3]:
lyrics = pd.read_csv("/kaggle/input/scrapped-lyrics-from-6-genres/lyrics-data.csv")
lyrics = lyrics[lyrics['language']=='en']
lyrics

,ALink,SName,SLink,Lyric,language
69,/ivete-sangalo/,Careless Whisper,/ivete-sangalo/careless-whisper.html,I feel so unsure\nAs I take your hand and lead...,en
86,/ivete-sangalo/,Could You Be Loved / Citação Musical do Rap: S...,/ivete-sangalo/could-you-be-loved-citacao-musi...,"Don't let them fool, ya\nOr even try to school...",en
88,/ivete-sangalo/,Cruisin' (Part. Saulo),/ivete-sangalo/cruisin-part-saulo.html,"Baby, let's cruise, away from here\nDon't be c...",en
111,/ivete-sangalo/,Easy,/ivete-sangalo/easy.html,"Know it sounds funny\nBut, I just can't stand ...",en
140,/ivete-sangalo/,For Your Babies (The Voice cover),/ivete-sangalo/for-your-babies-the-voice-cover...,You've got that look again\nThe one I hoped I ...,en
...,...,...,...,...,...
379926,/clegg-johnny/,The Waiting,/clegg-johnny/the-waiting.html,Chorus\nHere we stand waiting on the plain\nDa...,en
379927,/clegg-johnny/,Too Early For The Sky,/clegg-johnny/too-early-for-the-sky.html,I nearly disappeared into the mouth of a croco...,en
379928,/clegg-johnny/,Warsaw 1943 (I Never Betrayed The Revolution),/clegg-johnny/warsaw-1943-i-never-betrayed-the...,"Amambuka, amambuka azothengisa izwe lakithi, i...",en
379929,/clegg-johnny/,When The System Has Fallen,/clegg-johnny/when-the-system-has-fallen.html,Sweat in the heat for days on end\nwaiting for...,en


In [4]:
artists = pd.read_csv("/kaggle/input/scrapped-lyrics-from-6-genres/artists-data.csv")
artists = artists[(artists['Genres'].isin(['Rock'])) & (artists['Popularity']>5)]
df = lyrics.merge(artists[['Artist', 'Genres','Link']], left_on='ALink', right_on='Link', how='inner')


In [5]:
df

,ALink,SName,SLink,Lyric,language,Artist,Genres,Link
0,/4-non-blondes/,What's Up,/4-non-blondes/whats-up.html,Twenty-five years and my life is still\nTrying...,en,4 Non Blondes,Rock,/4-non-blondes/
1,/4-non-blondes/,Spaceman,/4-non-blondes/spaceman.html,Starry night bring me down\nTill I realize the...,en,4 Non Blondes,Rock,/4-non-blondes/
2,/4-non-blondes/,Pleasantly Blue,/4-non-blondes/pleasantly-blue.html,Every time you wake in the mornin'\nAnd you st...,en,4 Non Blondes,Rock,/4-non-blondes/
3,/4-non-blondes/,I'm The One,/4-non-blondes/im-the-one.html,Ah-hah!\nWoo!\nAh-ha-ha-ha-ha-ha!\nWe came her...,en,4 Non Blondes,Rock,/4-non-blondes/
4,/4-non-blondes/,Dear Mr. President,/4-non-blondes/dear-mr-president.html,I'm looking outside of my windows\nThe view th...,en,4 Non Blondes,Rock,/4-non-blondes/
...,...,...,...,...,...,...,...,...
949,/bruce-springsteen/,You've Got It,/bruce-springsteen/youve-got-it.html,"No one ever found it, ain't no school ever tau...",en,Bruce Springsteen,Rock,/bruce-springsteen/
950,/bruce-springsteen/,Youngstown,/bruce-springsteen/youngstown.html,Here in northeast Ohio\nBack in eighteen-o-thr...,en,Bruce Springsteen,Rock,/bruce-springsteen/
951,/bruce-springsteen/,Your Love,/bruce-springsteen/your-love.html,She promises to be yours forever and always\nA...,en,Bruce Springsteen,Rock,/bruce-springsteen/
952,/bruce-springsteen/,Your Own Worst Enemy,/bruce-springsteen/your-own-worst-enemy.html,Your Own Worst Enemy\n\n\nYou can't sleep at n...,en,Bruce Springsteen,Rock,/bruce-springsteen/


In [6]:
df = df.drop(columns=['ALink','SLink','Link'])
df = df[df['Lyric'].apply(lambda x: len(x.split(' ')) < 350)]

In [7]:
df

,SName,Lyric,language,Artist,Genres
0,What's Up,Twenty-five years and my life is still\nTrying...,en,4 Non Blondes,Rock
1,Spaceman,Starry night bring me down\nTill I realize the...,en,4 Non Blondes,Rock
2,Pleasantly Blue,Every time you wake in the mornin'\nAnd you st...,en,4 Non Blondes,Rock
3,I'm The One,Ah-hah!\nWoo!\nAh-ha-ha-ha-ha-ha!\nWe came her...,en,4 Non Blondes,Rock
4,Dear Mr. President,I'm looking outside of my windows\nThe view th...,en,4 Non Blondes,Rock
...,...,...,...,...,...
949,You've Got It,"No one ever found it, ain't no school ever tau...",en,Bruce Springsteen,Rock
950,Youngstown,Here in northeast Ohio\nBack in eighteen-o-thr...,en,Bruce Springsteen,Rock
951,Your Love,She promises to be yours forever and always\nA...,en,Bruce Springsteen,Rock
952,Your Own Worst Enemy,Your Own Worst Enemy\n\n\nYou can't sleep at n...,en,Bruce Springsteen,Rock


In [8]:
test_set = df.sample(n = 200)
test_set

,SName,Lyric,language,Artist,Genres
664,Johnny 99,Well they closed down the auto plant in Mahwah...,en,Bruce Springsteen,Rock
914,Walking Through Midnight,Well I've been walking through midnight\nAnd t...,en,Bruce Springsteen,Rock
110,Cracked Actor,I've come on a few years from my Hollywood Hig...,en,David Bowie,Rock
811,Silver Palomino,"A mother dies, leaving her young son to come t...",en,Bruce Springsteen,Rock
435,Who Can I Be Now,Everybody's raised in blindness\nEverbody know...,en,David Bowie,Rock
...,...,...,...,...,...
452,Dancing In The Dark,I get up in the evening\nAnd I ain't got nothi...,en,Bruce Springsteen,Rock
530,Candy's Room,In Candy's room there are pictures\nOf her her...,en,Bruce Springsteen,Rock
540,City Of Night,"Taxi Cab, Taxi Cab, at the light\nWon't you ta...",en,Bruce Springsteen,Rock
288,Please Mr. Gravedigger,There's a little churchyard just along the way...,en,David Bowie,Rock


In [9]:
df = df.loc[~df.index.isin(test_set.index)]
df

,SName,Lyric,language,Artist,Genres
1,Spaceman,Starry night bring me down\nTill I realize the...,en,4 Non Blondes,Rock
2,Pleasantly Blue,Every time you wake in the mornin'\nAnd you st...,en,4 Non Blondes,Rock
3,I'm The One,Ah-hah!\nWoo!\nAh-ha-ha-ha-ha-ha!\nWe came her...,en,4 Non Blondes,Rock
4,Dear Mr. President,I'm looking outside of my windows\nThe view th...,en,4 Non Blondes,Rock
5,Train,What ya gonna do child\nWhen your thoughts are...,en,4 Non Blondes,Rock
...,...,...,...,...,...
949,You've Got It,"No one ever found it, ain't no school ever tau...",en,Bruce Springsteen,Rock
950,Youngstown,Here in northeast Ohio\nBack in eighteen-o-thr...,en,Bruce Springsteen,Rock
951,Your Love,She promises to be yours forever and always\nA...,en,Bruce Springsteen,Rock
952,Your Own Worst Enemy,Your Own Worst Enemy\n\n\nYou can't sleep at n...,en,Bruce Springsteen,Rock


In [10]:
test_set = test_set.reset_index()
df = df.reset_index()

In [11]:
test_set

,index,SName,Lyric,language,Artist,Genres
0,664,Johnny 99,Well they closed down the auto plant in Mahwah...,en,Bruce Springsteen,Rock
1,914,Walking Through Midnight,Well I've been walking through midnight\nAnd t...,en,Bruce Springsteen,Rock
2,110,Cracked Actor,I've come on a few years from my Hollywood Hig...,en,David Bowie,Rock
3,811,Silver Palomino,"A mother dies, leaving her young son to come t...",en,Bruce Springsteen,Rock
4,435,Who Can I Be Now,Everybody's raised in blindness\nEverbody know...,en,David Bowie,Rock
...,...,...,...,...,...,...
195,452,Dancing In The Dark,I get up in the evening\nAnd I ain't got nothi...,en,Bruce Springsteen,Rock
196,530,Candy's Room,In Candy's room there are pictures\nOf her her...,en,Bruce Springsteen,Rock
197,540,City Of Night,"Taxi Cab, Taxi Cab, at the light\nWon't you ta...",en,Bruce Springsteen,Rock
198,288,Please Mr. Gravedigger,There's a little churchyard just along the way...,en,David Bowie,Rock


In [12]:
test_set = test_set.drop(columns = "index")
test_set

,SName,Lyric,language,Artist,Genres
0,Johnny 99,Well they closed down the auto plant in Mahwah...,en,Bruce Springsteen,Rock
1,Walking Through Midnight,Well I've been walking through midnight\nAnd t...,en,Bruce Springsteen,Rock
2,Cracked Actor,I've come on a few years from my Hollywood Hig...,en,David Bowie,Rock
3,Silver Palomino,"A mother dies, leaving her young son to come t...",en,Bruce Springsteen,Rock
4,Who Can I Be Now,Everybody's raised in blindness\nEverbody know...,en,David Bowie,Rock
...,...,...,...,...,...
195,Dancing In The Dark,I get up in the evening\nAnd I ain't got nothi...,en,Bruce Springsteen,Rock
196,Candy's Room,In Candy's room there are pictures\nOf her her...,en,Bruce Springsteen,Rock
197,City Of Night,"Taxi Cab, Taxi Cab, at the light\nWon't you ta...",en,Bruce Springsteen,Rock
198,Please Mr. Gravedigger,There's a little churchyard just along the way...,en,David Bowie,Rock


In [13]:
df

,index,SName,Lyric,language,Artist,Genres
0,1,Spaceman,Starry night bring me down\nTill I realize the...,en,4 Non Blondes,Rock
1,2,Pleasantly Blue,Every time you wake in the mornin'\nAnd you st...,en,4 Non Blondes,Rock
2,3,I'm The One,Ah-hah!\nWoo!\nAh-ha-ha-ha-ha-ha!\nWe came her...,en,4 Non Blondes,Rock
3,4,Dear Mr. President,I'm looking outside of my windows\nThe view th...,en,4 Non Blondes,Rock
4,5,Train,What ya gonna do child\nWhen your thoughts are...,en,4 Non Blondes,Rock
...,...,...,...,...,...,...
700,949,You've Got It,"No one ever found it, ain't no school ever tau...",en,Bruce Springsteen,Rock
701,950,Youngstown,Here in northeast Ohio\nBack in eighteen-o-thr...,en,Bruce Springsteen,Rock
702,951,Your Love,She promises to be yours forever and always\nA...,en,Bruce Springsteen,Rock
703,952,Your Own Worst Enemy,Your Own Worst Enemy\n\n\nYou can't sleep at n...,en,Bruce Springsteen,Rock


In [14]:
df = df.drop(columns = "index")
df

,SName,Lyric,language,Artist,Genres
0,Spaceman,Starry night bring me down\nTill I realize the...,en,4 Non Blondes,Rock
1,Pleasantly Blue,Every time you wake in the mornin'\nAnd you st...,en,4 Non Blondes,Rock
2,I'm The One,Ah-hah!\nWoo!\nAh-ha-ha-ha-ha-ha!\nWe came her...,en,4 Non Blondes,Rock
3,Dear Mr. President,I'm looking outside of my windows\nThe view th...,en,4 Non Blondes,Rock
4,Train,What ya gonna do child\nWhen your thoughts are...,en,4 Non Blondes,Rock
...,...,...,...,...,...
700,You've Got It,"No one ever found it, ain't no school ever tau...",en,Bruce Springsteen,Rock
701,Youngstown,Here in northeast Ohio\nBack in eighteen-o-thr...,en,Bruce Springsteen,Rock
702,Your Love,She promises to be yours forever and always\nA...,en,Bruce Springsteen,Rock
703,Your Own Worst Enemy,Your Own Worst Enemy\n\n\nYou can't sleep at n...,en,Bruce Springsteen,Rock


In [15]:
test_set['true_end'] = test_set['Lyric'].str.split().str[-20:].apply(' '.join)
test_set['Lyric'] = test_set['Lyric'].str.split().str[:-20].apply(' '.join)

In [16]:
test_set

,SName,Lyric,language,Artist,Genres,true_end
0,Johnny 99,Well they closed down the auto plant in Mahwah...,en,Bruce Springsteen,Rock,it over judge one more time And let `em shave ...
1,Walking Through Midnight,Well I've been walking through midnight And th...,en,Bruce Springsteen,Rock,stumbling blind With no destination At an unho...
2,Cracked Actor,I've come on a few years from my Hollywood Hig...,en,David Bowie,Rock,dead Oh yeah Oooh stay For a day Oh yeah Don't...
3,Silver Palomino,"A mother dies, leaving her young son to come t...",en,Bruce Springsteen,Rock,scrub pine that grows I watch the silver palom...
4,Who Can I Be Now,Everybody's raised in blindness Everbody know'...,en,David Bowie,Rock,found love Can I be real Ye ye yeah Can I be r...
...,...,...,...,...,...,...
195,Dancing In The Dark,I get up in the evening And I ain't got nothin...,en,Bruce Springsteen,Rock,This gun's for hire Even if we're just dancing...
196,Candy's Room,In Candy's room there are pictures Of her hero...,en,Bruce Springsteen,Rock,give all that I got to give All that I want al...
197,City Of Night,"Taxi Cab, Taxi Cab, at the light Won't you tak...",en,Bruce Springsteen,Rock,"brings it with her man when she comes Hey, hey..."
198,Please Mr. Gravedigger,There's a little churchyard just along the way...,en,David Bowie,Rock,"know of it Hello misses, thought she'd be a li..."


In [19]:
class SongLyrics(Dataset):
    def __init__(self, control_code, truncate = False, gpt2_type = "gpt2", max_length = 1024):
        
        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []
        
        for row in df["Lyric"]:
            self.lyrics.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")))
            
        if truncate:
            self.lyrics = self.lyrics[:20000]
                
        self.lyrics_count = len(self.lyrics) 
            
    def __len__(self):
        return self.lyrics_count 
    
    def __getitem__(self, item):
        return self.lyrics[item]
    
    
dataset = SongLyrics(df["Lyric"], truncate = True, gpt2_type= "gpt2")


In [20]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [21]:
# To combine multiple tensors into a single packed tensor

def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [22]:
from tqdm import tqdm, trange

In [23]:
# Accumulated batch is used for keeping track of the number of processed batches
# "accumulating_batch_count % batch_size) == 0" means the optimization step will be performed
# once every "batch_size" processed batches.

def train(
    dataset, model, tokenizer, batch_size=16, epochs=20, lr=2e-5, 
    warmup_steps = 200, max_seq_len=400, gpt2_type = "gpt2",
    test_mode = False,
):
    acc_steps = 100
    device = torch.device("cuda")
    model = model.cuda()
    
    model.train()
    
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,num_warmup_steps=warmup_steps, num_training_steps=-1)
    
    train_dataloader = DataLoader(dataset, batch_size = 1, shuffle=True)
    loss = 0
    accumulating_batch_count = 0
    input_tensor = None
    
    for epoch in range(epochs):
        print(f"Training epoch {epoch}")
        print(f"Loss at epoch {epoch} is {loss}")
        
        for idx,entry in tqdm(enumerate(train_dataloader)):
            
            entry = entry.to(device)
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)
            
            
            if carry_on and idx != len(train_dataloader) - 1:
                continue
                
            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()
            
            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()
                
            accumulating_batch_count +=1
            input_tensor = None
            
    return model


In [24]:
model = train(dataset, model, tokenizer)

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Training epoch 0
Loss at epoch 0 is 0


705it [00:57, 12.31it/s]


Training epoch 1
Loss at epoch 1 is 2.776547431945801


705it [00:58, 12.05it/s]


...

705it [01:00, 11.73it/s]


Training epoch 18
Loss at epoch 18 is 1.9264748096466064


705it [01:00, 11.70it/s]


Training epoch 19
Loss at epoch 19 is 2.1653995513916016


705it [01:00, 11.73it/s]


In [25]:
torch.save(model.state_dict(),os.path.join("/kaggle/working/", "model.pt"))
torch.save(tokenizer,os.path.join("/kaggle/working/", "tokenizer.pt"))

In [26]:
mask = test_set["Lyric"].eq("")
empty_rows = test_set[mask]
empty_rows

,SName,Lyric,language,Artist,Genres,true_end
100,Just For One Day (Heroes David Guetta Remix),,en,David Bowie,Rock,We can be Heroes Just for one day


In [27]:
empty_indices = empty_rows.index.tolist()
empty_indices

[100]

In [28]:
test_set = test_set.drop(index=empty_indices)
test_set = test_set.reset_index()

In [29]:
test_set

,index,SName,Lyric,language,Artist,Genres,true_end
0,0,Johnny 99,Well they closed down the auto plant in Mahwah...,en,Bruce Springsteen,Rock,it over judge one more time And let `em shave ...
1,1,Walking Through Midnight,Well I've been walking through midnight And th...,en,Bruce Springsteen,Rock,stumbling blind With no destination At an unho...
2,2,Cracked Actor,I've come on a few years from my Hollywood Hig...,en,David Bowie,Rock,dead Oh yeah Oooh stay For a day Oh yeah Don't...
3,3,Silver Palomino,"A mother dies, leaving her young son to come t...",en,Bruce Springsteen,Rock,scrub pine that grows I watch the silver palom...
4,4,Who Can I Be Now,Everybody's raised in blindness Everbody know'...,en,David Bowie,Rock,found love Can I be real Ye ye yeah Can I be r...
...,...,...,...,...,...,...,...
194,195,Dancing In The Dark,I get up in the evening And I ain't got nothin...,en,Bruce Springsteen,Rock,This gun's for hire Even if we're just dancing...
195,196,Candy's Room,In Candy's room there are pictures Of her hero...,en,Bruce Springsteen,Rock,give all that I got to give All that I want al...
196,197,City Of Night,"Taxi Cab, Taxi Cab, at the light Won't you tak...",en,Bruce Springsteen,Rock,"brings it with her man when she comes Hey, hey..."
197,198,Please Mr. Gravedigger,There's a little churchyard just along the way...,en,David Bowie,Rock,"know of it Hello misses, thought she'd be a li..."


In [30]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, 
    top_p=0.8,
    temperature=1.,
):
    device = torch.device("cuda")
    model = model.cuda()
    
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = float("-inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                generated = generated.to(device)
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
                
                
                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0 

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().cpu().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
                output_list = list(generated.squeeze().cpu().numpy())
                output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
                generated_list.append(output_text)
                
    return generated_list

def text_generation(test_data):
    generated_lyrics = []
    for i in range(len(test_data)):
        x = generate(model, tokenizer, test_data['Lyric'][i], entry_count=1)
        generated_lyrics.append(x)
    return generated_lyrics

generated_lyrics = text_generation(test_set)


100%|██████████| 1/1 [00:00<00:00,  1.56it/s]


In [32]:
generated_lyrics[0]

['Well they closed down the auto plant in Mahwah late that month Ralph went out lookin\' for a job but he couldn\'t find none He came home too drunk from mixin\' Tanqueray and wine He got a gun shot a night clerk now they call\'m Johnny 99 Down in the part of town where when you hit a red light you don\'t stop Johnny\'s wavin\' his gun around and threatenin\' to blow his top When an off-duty cop snuck up on him from behind Out in front of the Club Tip Top they slapped the cuffs on Johnny 99 Well the city supplied a public defender but the judge was Mean John Brown He came into the courtroom and stared young Johnny down Well the evidence is clear gonna let the sentence son fit the crime Prison for 98 and a year and we\'ll call it even Johnny 99 A fist fight broke out in the courtroom they had to drag Johnny\'s girl away His mama stood up and shouted "Judge don\'t take my boy this way" Well son you got a statement you\'d like to make Before the bailiff comes to forever take you away Now 

In [37]:
test_set["Lyric"][0]

'Well they closed down the auto plant in Mahwah late that month Ralph went out lookin\' for a job but he couldn\'t find none He came home too drunk from mixin\' Tanqueray and wine He got a gun shot a night clerk now they call\'m Johnny 99 Down in the part of town where when you hit a red light you don\'t stop Johnny\'s wavin\' his gun around and threatenin\' to blow his top When an off-duty cop snuck up on him from behind Out in front of the Club Tip Top they slapped the cuffs on Johnny 99 Well the city supplied a public defender but the judge was Mean John Brown He came into the courtroom and stared young Johnny down Well the evidence is clear gonna let the sentence son fit the crime Prison for 98 and a year and we\'ll call it even Johnny 99 A fist fight broke out in the courtroom they had to drag Johnny\'s girl away His mama stood up and shouted "Judge don\'t take my boy this way" Well son you got a statement you\'d like to make Before the bailiff comes to forever take you away Now j

In [36]:
len(generated_lyrics)

199

In [61]:
my_generations = []

for i in range(len(generated_lyrics)):
    a = test_set["Lyric"][i].split()[-30:]
    b = " ".join(a)
    c = " ".join(generated_lyrics[i])
    my_generations.append(c.split(b)[-1])
    
my_generations[0]


" of the worst thing you've ever seen Don't let the boy rob you of a dime You said the line at Wal-Mart got up every morning<|endoftext|>"

In [66]:
for i in range(len(my_generations)):
    my_generations[i] = my_generations[i][:-13]
    
my_generations[:2]

[" of the worst thing you've ever seen Don't let the boy rob you of a dime You said the line at Wal-Mart got up every morning",
 " standing in front of my fire's doorway I'm thinking, as if I'm dying It's over with But it's lost in my mind I don"]

In [67]:
len(my_generations)

199

In [68]:
test_set["generated_lyrics"] = my_generations
test_set

,index,SName,Lyric,language,Artist,Genres,true_end,generated_lyrics
0,0,Johnny 99,Well they closed down the auto plant in Mahwah...,en,Bruce Springsteen,Rock,it over judge one more time And let `em shave ...,of the worst thing you've ever seen Don't let...
1,1,Walking Through Midnight,Well I've been walking through midnight And th...,en,Bruce Springsteen,Rock,stumbling blind With no destination At an unho...,standing in front of my fire's doorway I'm th...
2,2,Cracked Actor,I've come on a few years from my Hollywood Hig...,en,David Bowie,Rock,dead Oh yeah Oooh stay For a day Oh yeah Don't...,dead Oh stay Please stay Please stay\n\nRead ...
3,3,Silver Palomino,"A mother dies, leaving her young son to come t...",en,Bruce Springsteen,Rock,scrub pine that grows I watch the silver palom...,rush your love lingers
4,4,Who Can I Be Now,Everybody's raised in blindness Everbody know'...,en,David Bowie,Rock,found love Can I be real Ye ye yeah Can I be r...,was my brother\n\nAnd so he started running i...
...,...,...,...,...,...,...,...,...
194,195,Dancing In The Dark,I get up in the evening And I ain't got nothin...,en,Bruce Springsteen,Rock,This gun's for hire Even if we're just dancing...,on your side And if it comes back it's gonna ...
195,196,Candy's Room,In Candy's room there are pictures Of her hero...,en,Bruce Springsteen,Rock,give all that I got to give All that I want al...,better stay she knows that I'll be forever an...
196,197,City Of Night,"Taxi Cab, Taxi Cab, at the light Won't you tak...",en,Bruce Springsteen,Rock,"brings it with her man when she comes Hey, hey...","stays true to her promises Ah hey hey, ah hey..."
197,198,Please Mr. Gravedigger,There's a little churchyard just along the way...,en,David Bowie,Rock,"know of it Hello misses, thought she'd be a li...","care I'm a little child, but she is so very k..."


In [72]:
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores = []

for i in range(len(test_set)):
    reference = test_set["true_end"][i]
    candidate = test_set["generated_lyrics"][i]
    scores.append(sentence_bleu(reference, candidate))
    
statistics.mean(scores)

1.1592545031428148e-231

In [73]:
test_set["Lyric"][5]

"Living in lies by the railway line Pushing the hair from my eyes Elvis is English and climbs the hills Can't tell the bullshit from the lies Screaming along in South London Vicious but ready to learn Sometimes I fear that the whole world is queer Sometimes but always in vain So I'll wait until we're sane Wait until we're blessed and all the same Full of blood, loving life and all it's got to give Englishmen going insane Down on my knees in suburbia Down on myself in every way With great expectations I change all my clothes Mustn't crumble at silver and gold Screaming above Central London Never born, so I'll never get old So I'll wait until we're sane Wait until we're blessed and all the same Full of blood, loving life and all it's got to give Englishmen going insane Down on my knees in suburbia Down on myself in every way Day after Day after"

In [74]:
test_set["true_end"][5]

'Day Day after Zane, Zane, Zane Ouvre le chien Day after Day Day after Zane, Zane, Zane Ouvre le chien'

In [75]:
test_set["generated_lyrics"][5]

" day I'm feeling so lost It's always like something's coming from heaven Inside my head, looking down at my watch There's something..."

In [76]:
test_set["Lyric"][42]

"Back into the funhouse Music is sublime See the lady's spinning She'll show me We all lie in the funhouse No one is your friend No one loves a loser We love you In the funhouse In the funhouse In the funhouse In the funhouse In the funhouse In the funhouse We'll show you Really good time We all lie in the funhouse Girls wag their pain No one sleeps in the funhouse They'll show you (Fun fun fun fun Fun fun fun fun) We'll show you (Fun fun fun fun Fun fun fun fun) In the funhouse In the funhouse In the funhouse We'll show you Really good time Back into the funhouse My dreamdogs have arrived My summer turns to fall And I'll"

In [77]:
test_set["true_end"][42]

"miss you We'll show you She'll show me We'll show you She'll show me Really good time Really good time"

In [78]:
test_set["generated_lyrics"][42]

' be your teddy bear My baby is yours to play with My baby is yours to play with\n1. Musicals and Performs.\nMus'

In [82]:
test_set["Lyric"][88]

'I remember yesterday and I’d sit and watch the hound dogs play Howlin’ at the china moon ? ? now yesterday is the busted blues Life was young, things were easy, Days were short, nights were warm Time were good. hate was shallow ? ? the horse crazy behind the ? ? ? ? I swear that I’ve seen your face somewhere back in that time And I’m in theat place. do you remember? The old house stood like world war two with just rooms And a hall to be used. The lady was mean. Just slightly unclean with a heart of cold silver and gold. Kitchen smelled of kerosene. Celing hung down on ? veil? and rotted beams And the man, they said, his work could of hung in the room Now he sits around all day because his left arm won’t move He was a master of the art of electricity He lectured on tours and circuitry. He was self-employed, but he could never see his way into the light He had a room full of switches and dials and lights and A head full of clouds and eyes full of sight. And when it got dark, I could hea

In [83]:
test_set["true_end"][88]

'sfternoon watching All my cartoons thru the hall And across the porch was the sun surrendered like a crying torch.'

In [84]:
test_set["generated_lyrics"][88]

' day playing games and watching my boy play I decided to put it on and put the number down just for the kids! Who were we gonna watch today'

In [96]:
test_set["Lyric"][163]

"I'll give you back my right to be free I'll give you back my farm and I'll give you back my house I'll give you back my right to be free I've got moon, I've got sun Lightning, lightning, lightning I've got me, I've got you Frightening, frightening, frightening I'll give you back my farmland and I'll give you back my house I'll give you back my right to be free I'll give you back my farmland and I'll give you back my house I'll give you back my right to be free I've"

In [97]:
test_set["true_end"][163]

"got moon, I've got sun (I've got) Lightning, lightning, lightning I've got me, I've got you (It's) Frightening, frightening, frightening"

In [98]:
test_set["generated_lyrics"][163]

" got treasure, treasure, treasure I've got treasure, treasure I've got me I've got treasure, treasure, treasure I've got me I've"